# Multi-scenario multiobjective optimization problems

The DESDEO framework supports handling multiple scenarios in multiobjective optimization problems applicable in scenario planning and decision-making under deep uncertainty. Indeed, these problems can be formulated as a multiobjective optimization problem where an objective function is a combination of scenarios and objectives called a meta-objective function. Then, one can use most of the available solution methods in DESDEO to solve the formulated problem. 

Here, we want to show how a multi-scenario multiobjective optimization problem can be added and solved with available interactive multiobjective methods in DESDEO.

We consider the following problem:

**Problem description:**

The river pollution problem [1] considers a river close to a city. 
There are two sources of pollution: industrial pollution from a 
fishery and municipal waste from the city and two treatment plants 
(in the fishery and the city). The pollution is reported in pounds of 
biochemical oxygen demanding material (BOD), and water quality is 
measured in dissolved oxygen concentration (DO).

Cleaning water in the city increases tax rate, and cleaning in the 
fishery reduces the return on investment. The problem is to improve 
the DO level in the city and at the municipality border (f1 and f2, respectively)
while, at the same time, maximizing the percent return on investment at the fishery (f3)
and minimizing addition to the city tax (f4).

Decision variables are:

* $ x_1 $:the proportional amounts of BOD removed from water after the fishery(treatment plant1)
* $ x_2 $:the proportional amounts of BOD removed from water after the city(treatment plant2)


The original problem [1] considered certain values for all the parameters. In this paper, we assume some of the parameters are deeply uncertain, and only a range of plausible values is known for each. These deeply uncertain parameters are as follows:

* $ a (\alpha) \in [3, 4.24]$, Water quality index after the fishery.

* $ b (\beta) \in [2.25, 2.4]$, BOD reduction rate at treatment plant 1 (after fishery).

* $ d (\delta) \in [0.075, 0.092]$, BOD reduction rate at treatment plant 2 (after the city).

* $ e (\xi) \in [0.067, 0.083]$, The effective rate of BOD reduction at treatment plant 1 after the city.

* $ h (\eta)  \in [1.2, 1.50]$, Another parameter that uses to calculate the effective BOD reduction rate at the second treatment plant.

* $ m (r) \in [5.1, 12.5]$, Investment return rate.

Then, the uncertain version of the river problem has been formulated as follows [2]:

$$
\begin{equation}\
\begin{array}{rll}\
\text{maximize}  & f_1({\mathbf{x}}) =& \alpha + (log((\frac{\beta}{2} - 1.14)^2) + \beta^3) x_1 \\ % - 4.07 - 2.27 x_1 
\text{maximize}  & f_2({\mathbf{x}}) =& \gamma + \delta x_1  + \xi x_2  + \frac{0.01}{\eta - x_1^2} + \frac{0.30}{\eta - x_2^2} \\ 
\text{maximize}  & f_3({\mathbf{x}}) =&  r - \frac{0.71}{1.09 - x_1^2} \\ 
\text{mininimize}  & f_4({\mathbf{x}}) =& - 0.96 + \frac{0.96}{1.09 - x_2^2} \\ 
\text{subject to}  && 0.3 \leq x_1, x_2 \leq 1.0, \\
\end{array}\
\end{equation}
$$
where $\gamma = log((\frac{\alpha}{2}-1)) + \frac{\alpha}{2} + 1.5$.


[1] S. C. Narula and H. R. Weistroffer, ‘‘A flexible method for nonlinear multicriteria decision-making problems,’’ IEEE Trans. Syst., Man, Cybern.,
vol. 19, no. 4, pp. 883–887, Jul. 1989.

[2] B. Shavazipour, J. H. Kwakkel, K. Miettinen, ‘‘Let decision-makers direct the search for robust solutions: An interactive framework for multiobjective robust optimization under deep uncertainty,’’ submitted.

Let us start with importing required objects and classes and intializing the problem.

In [8]:
import numpy as np
import pandas as pd

from desdeo_problem.problem.Variable import Variable
from desdeo_problem.problem.Objective import VectorObjective
from desdeo_problem.problem.Problem import MOProblem
from desdeo_mcdm.utilities import payoff_table_method

Now, we read previously selected scenarios to be considered in the multi-scenario multiobjective optimization problem (the selected scenarios are the same as the ones considered in [2]).

* Alternatively one can define/update them (as a PandaDataFrame) manually, by changing the values of deeply uncertain parameters in the ranges mentioened above.

In [9]:
scenarios = pd.read_csv(r'data/selected_scenarios_river(12.5.22).csv')
scenarios

,Scenario_num,a,b,c,d,e,h,m
0,1,4.070,2.270,3.600,0.0800,0.0750,1.39,8.21
1,2,3.868,2.262,3.700,0.0869,0.0782,1.47,10.28
2,3,3.620,2.278,3.324,0.0835,0.0750,1.23,5.84
3,4,3.372,2.254,4.076,0.0903,0.0814,1.35,11.76
4,5,3.124,2.270,3.888,0.0801,0.0686,1.29,7.32
5,6,4.116,2.286,3.512,0.0767,0.0718,1.41,8.80


Now, let us define the problem. Note that, in this example, the fourth objective function involved no uncertain parameter. Therefore, there is no variation in its values defining the problem. So, we should separate the uncertain and deterministic objective functions. 

In [10]:
n_uncertain_obj = 3
n_deterministic_obj = 1

Finally, we define the corresponding multi-scenario multiobjective optimization (MS-MOP) as follows:

* Note that, in MS-MOPs, we often consider each scenario-objective combination as a meta-objective. 
* Also, notice the argument of the callable objective function is assumed to be array-like.

In [11]:
def f_multi_scenarios(x: np.ndarray) -> np.ndarray:
        """
        Arguments:
        x (np.array): initial variable values. Must be between 0.3 and 1.0.
        Returns:
        output (np.array): objective values in all selected scenarios.
        """
        x = np.atleast_2d(x)
        output = np.zeros((n_uncertain_obj*len(scenarios)+n_deterministic_obj, len(x)))
        
        for s in range(len(scenarios)):
            a = scenarios.a[s]
            b = scenarios.b[s]
            d = scenarios.d[s]
            e = scenarios.e[s]
            h = scenarios.h[s]
            m = scenarios.m[s]
            # Definning the objective functions (negative sign at the beginning is to convert maximization to minimization).
            output[s*n_uncertain_obj+0,:] = - (a + (np.log((b/2-1.14)**2) + b**3) * x[:, 0])
            output[s*n_uncertain_obj+1,:] = - ((np.log((a/2-1)) + a/2 + 1.5) + d*x[:, 0] + e*x[:, 1] + 0.01 / (h - x[:, 0]**2) + 0.3 / (h - x[:, 1]**2))
            output[s*n_uncertain_obj+2,:] = - (m - 0.71 / (1.09 - x[:, 0]**2))

        # f_4 has no uncertain parameters and returning a certain value in all scenarios without any variation.
        # Multi
        output[-1,:] = - (0.96 - 0.96 / (1.09 - x[:, 1]**2))
    
        return output.T  #np.reshape(meta_objectives, (1,-1))


Define the variables and create the problem object:

In [12]:
# Initial variable values need to be between lower and upper bounds")      
x_1 = Variable("the proportionate amount of BOD removed from water at the fishery", 0.5, 0.3, 1.0)
x_2 = Variable("the proportionate amount of BOD removed from water at the city", 0.5, 0.3, 1.0)

variables = [x_1, x_2]

# Creating functions names
# The number of objective functions is varied based on the number of considered scenarios in the MS-MOP.
Y = []
for s in range(len(scenarios)):
    Y.extend([f"f_1{s+1}", f"f_2{s+1}",f"f_3{s+1}"])
Y.extend(["f4"])   
    
#
F = VectorObjective(Y, f_multi_scenarios)

problem = MOProblem(variables=variables, objectives=[F])


Now, the problem is fully specified and can be evaluated and played around with.

In [13]:
print("N of objectives:", problem.n_of_objectives)
print("N of variables:", problem.n_of_variables)
print("N of constraints:", problem.n_of_constraints)

N of objectives: 19
N of variables: 2
N of constraints: 0


In [14]:
f_multi_scenarios(np.array([.5, .5]))

array([[ -4.62022413,  -3.91883125,  -7.3647619 ,  -4.94439366,
         -3.70236952,  -9.4347619 ,  -2.6228392 ,  -3.4948555 ,
         -4.9947619 ,  -4.75493561,  -3.17679053, -10.9147619 ,
         -3.67422413,  -2.85817349,  -6.4747619 ,  -4.27994184,
         -3.95587171,  -7.9547619 ,   0.18285714]])

In [15]:
problem.evaluate([1, 1])

EvaluationResults(objectives=array([[-5.17044827, -4.51927322, -0.32111111, -6.02078732, -4.19039563,
        -2.39111111, -1.62567839, -4.60560506,  2.04888889, -6.13787122,
        -3.86653663, -3.87111111, -4.22444827, -3.70341209,  0.56888889,
        -4.44388368, -4.51897789, -0.91111111,  9.70666667]]), fitness=array([[-5.17044827, -4.51927322, -0.32111111, -6.02078732, -4.19039563,
        -2.39111111, -1.62567839, -4.60560506,  2.04888889, -6.13787122,
        -3.86653663, -3.87111111, -4.22444827, -3.70341209,  0.56888889,
        -4.44388368, -4.51897789, -0.91111111,  9.70666667]]), constraints=None, uncertainity=array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan]]))

In [16]:
problem.evaluate([.3, .3])

EvaluationResults(objectives=array([[ -4.40013448,  -3.85436297,  -7.5       ,  -4.5138362 ,
         -3.63988884,  -9.57      ,  -3.02170352,  -3.41875879,
         -5.13      ,  -4.20176137,  -3.10666409, -11.05      ,
         -3.45413448,  -2.7886899 ,  -6.61      ,  -4.2143651 ,
         -3.89377882,  -8.09      ,  -0.        ]]), fitness=array([[ -4.40013448,  -3.85436297,  -7.5       ,  -4.5138362 ,
         -3.63988884,  -9.57      ,  -3.02170352,  -3.41875879,
         -5.13      ,  -4.20176137,  -3.10666409, -11.05      ,
         -3.45413448,  -2.7886899 ,  -6.61      ,  -4.2143651 ,
         -3.89377882,  -8.09      ,  -0.        ]]), constraints=None, uncertainity=array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan]]))

We can also calculate the ideal and nadir vector using DESEO's payoff-table.

In [17]:

ideal, nadir = payoff_table_method(problem)

print("The ideal vector =", ideal)
print("The nadir vector =", nadir)

The ideal vector = [-5.17044717e+00 -4.51926899e+00 -7.49999957e+00 -6.02078517e+00
 -4.19039266e+00 -9.56999957e+00 -3.02170152e+00 -4.60559318e+00
 -5.12999957e+00 -6.13786846e+00 -3.86653140e+00 -1.10499996e+01
 -4.22444717e+00 -3.70340457e+00 -6.60999957e+00 -4.44388335e+00
 -4.51897406e+00 -8.08999957e+00  5.76214960e-07]
The nadir vector = [-4.40013558 -3.87415953 -0.32128642 -4.51383835 -3.65854407 -2.39128642
 -1.62568039 -3.44245614  2.04871358 -4.20176413 -3.12791133 -3.87128642
 -3.45413558 -2.81011838  0.56871358 -4.21436543 -3.91287893 -0.91128642
  9.70642964]
